# 2023

In [19]:
import re
from operator import mul
from queue import Queue
import numpy as np 

In [51]:
def j1():
    filename = "Input/inputj1.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        total = 0
        for line in input:
            line = re.sub('\D', '', line)
            nums = int(line[0] + line[-1])
            total += nums
        print(f"Réponse jour 1 partie 1 : {total}")
    
    def partie2():
        values = {
            "one": "1", 
            "two": "2", 
            "three": "3", 
            "four": "4", 
            "five": "5", 
            "six": "6", 
            "seven": "7", 
            "eight": "8", 
            "nine": "9"
            }
        pairs = []
        for line in input:
            digits = []
            for i,c in enumerate(line):
                if line[i].isdigit():
                    digits.append(line[i])
                else:
                    for k in values.keys():
                        if line[i:].startswith(k):
                            digits.append(values[k])
            pairs.append(int(f"{digits[0]}{digits[-1]}"))
        
        print(f"Réponse jour 1 partie 2 : {sum(pairs)}")
    
    partie1()
    partie2()

j1()

Réponse jour 1 partie 1 : 54304
Réponse jour 1 partie 2 : 54418


In [52]:
def j2():
    filename = "Input/inputj2.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        gameID = 0
        gameIDSum = 0
        reds = 12
        greens = 13
        blues = 14
        gameIDRegex = "(\\d+(?=:))"
        setsRegex = "(?<=:)(.*)"
        redsRegex = "(\\d+)(?=\\sred)"
        greensRegex = "(\\d+)(?=\\sgreen)"
        bluesRegex = "(\\d+)(?=\\sblue)"

        for line in input:
            valid = True
            gameID = int(re.search(gameIDRegex, line).group(0))
            sets = re.search(setsRegex, line).group(0)
            sets = re.split(";", sets)
            for set in sets:
                currRed = re.search(redsRegex, set)
                if(currRed):
                    currRed = int(currRed.group(0))
                    if(currRed > reds): valid = False

                currGreen = re.search(greensRegex, set)
                if(currGreen):
                    currGreen = int(currGreen.group(0))
                    if(currGreen > greens): valid = False


                currBlue = re.search(bluesRegex, set)
                if(currBlue):
                    currBlue = int(currBlue.group(0))
                    if(currBlue > blues): valid = False
                    
            if(valid): gameIDSum += gameID

        print(f"Réponse jour 2 partie 1 : {str(gameIDSum)}")
    
    def partie2():
        sumOfPowers = 0
        setsRegex = "(?<=:)(.*)"
        redsRegex = "(\\d+)(?=\\sred)"
        greensRegex = "(\\d+)(?=\\sgreen)"
        bluesRegex = "(\\d+)(?=\\sblue)"

        for line in input:
            redMax = 0
            greenMax = 0
            blueMax = 0
            sets = re.search(setsRegex, line).group(0)
            sets = re.split(";", sets)
            for set in sets:
                currRed = re.search(redsRegex, set)
                if(currRed):
                    currRed = int(currRed.group(0))
                    if(currRed > redMax): redMax = currRed

                currGreen = re.search(greensRegex, set)
                if(currGreen):
                    currGreen = int(currGreen.group(0))
                    if(currGreen > greenMax): greenMax = currGreen

                currBlue = re.search(bluesRegex, set)
                if(currBlue):
                    currBlue = int(currBlue.group(0))
                    if(currBlue > blueMax): blueMax = currBlue
            power = redMax * greenMax * blueMax
            sumOfPowers += power

        print(f"Réponse jour 2 partie 2 : {str(sumOfPowers)}")
    
    partie1()
    partie2()
    
j2()

Réponse jour 2 partie 1 : 2101
Réponse jour 2 partie 2 : 58269


In [53]:
def j3():
    filename = "Input/inputj3.txt"
        
    with open(filename, 'r') as f:
        input = f.read()
        
    def partie1():
        lines = input.split('\n')

        symbol_regex = r'[^.\d]'
        symbol_adjacent = set()
        for i, line in enumerate(lines):
            for m in re.finditer(symbol_regex, line):
                j = m.start()
                symbol_adjacent |= {(r, c) for r in range(i-1, i+2) for c in range(j-1, j+2)}

        number_regex = r'\d+'
        part_num_sum = 0
        for i, line in enumerate(lines):
            for m in re.finditer(number_regex, line):
                if any((i, j) in symbol_adjacent for j in range(*m.span())):
                    part_num_sum += int(m.group())

        print(f"Réponse jour 3 partie 1 : {part_num_sum}")
    
    def partie2():
        lines = input.split('\n')

        gear_regex = r'\*'
        gears = dict()
        for i, line in enumerate(lines):
            for m in re.finditer(gear_regex, line):
                gears[(i, m.start())] = []

        number_regex = r'\d+'
        for i, line in enumerate(lines):
            for m in re.finditer(number_regex, line):
                for r in range(i-1, i+2):
                    for c in range(m.start()-1, m.end()+1):
                        if (r, c) in gears:
                            gears[(r, c)].append(int(m.group()))

        gear_ratio_sum = 0
        for nums in gears.values():
            if len(nums) == 2:
                gear_ratio_sum += mul(*nums)

        print(f"Réponse jour 3 partie 2 : {gear_ratio_sum}")
    
    partie1()
    partie2()
    
j3()

Réponse jour 3 partie 1 : 557705
Réponse jour 3 partie 2 : 84266818


In [96]:
def j4():
    filename = "Input/inputj4.txt"
    with open(filename, 'r') as f:
        input = f.read()
        
    def partie1():
        regex = r':(.*)\|(.*)'
        points = 0
        for win_nums, true_nums in re.findall(regex, input):
            overlap = set(win_nums.split()) & set(true_nums.split())
            if overlap:
                points += 2 ** (len(overlap) - 1)

        print(f"Réponse jour 4 partie 1 : {points}")
    
    def partie2():
        lines = []
        with open(filename) as file:
            lines = [line.strip() for line in file.readlines()]
        ans = 0
        d = {}
        q = Queue()

        for line in lines:
            cid, numbers = line.split(':')
            cid = int(cid[4:])
            winner, ours = numbers.split('|')
            ws = set(winner.split())
            os = set(ours.split())
            matches = ws & os

            # for each card, we just need to know how many new cards we add to the queue
            d[cid] = len(matches)
            # and fill the queue with the initial cards
            q.put(cid)


        # we proccess the queue until it's empty
        while not q.empty():
            ans += 1
            k = q.get()
            # and for each card, we add how many matches it had
            # starting from it's id (k) + 1
            for i in range(k + 1, k + d[k] + 1):
                q.put(i)
        
        print(f"Réponse jour 4 partie 2 : {ans}")
    
    partie1()
    partie2()
    
j4()

Réponse jour 4 partie 1 : 18653
Réponse jour 4 partie 2 : 5921508


In [5]:
def j5():
    filename = "Input/inputj5.txt"
    with open(filename, 'r') as f:
        input = f.read()
        
    def partie1():
        segments = input.split('\n\n')
        seeds = re.findall(r'\d+', segments[0])

        min_location = float('inf')
        for x in map(int, seeds):
            for seg in segments[1:]:
                for conversion in re.findall(r'(\d+) (\d+) (\d+)', seg):
                    destination, start, delta = map(int, conversion)
                    if x in range(start, start + delta):
                        x += destination - start
                        break

            min_location = min(x, min_location)
        print(f"Réponse jour 5 partie 1 : {min_location}")
    
    def partie2():
        segments = input.split('\n\n')
        intervals = []

        for seed in re.findall(r'(\d+) (\d+)', segments[0]):
            x1, dx = map(int, seed)
            x2 = x1 + dx
            intervals.append((x1, x2, 1))

        min_location = float('inf')
        while intervals:
            x1, x2, level = intervals.pop()
            if level == 8:
                min_location = min(x1, min_location)
                continue

            for conversion in re.findall(r'(\d+) (\d+) (\d+)', segments[level]):
                z, y1, dy = map(int, conversion)
                y2 = y1 + dy
                diff = z - y1
                if x2 <= y1 or y2 <= x1:    # no overlap
                    continue
                if x1 < y1:                 # split original interval at y1
                    intervals.append((x1, y1, level))
                    x1 = y1
                if y2 < x2:                 # split original interval at y2
                    intervals.append((y2, x2, level))
                    x2 = y2
                intervals.append((x1 + diff, x2 + diff, level + 1)) # perfect overlap -> make conversion and let pass to next nevel 
                break

            else:
                intervals.append((x1, x2, level + 1))
        print(f"Réponse jour 5 partie 2 : {min_location}")
    
    partie1()
    partie2()
    
j5()

Réponse jour 5 partie 1 : 486613012
Réponse jour 5 partie 2 : 56931769


In [18]:
def j6():
    filename = "Input/inputj6.txt"
    def partie1():
        def read_input(puzzle):
            with open(filename, 'r') as f:
                data = f.read().splitlines()
            return data
        
        def get_distance_travelled(pair):
            # exclude 0 and max value as it never makes sense
            # start the counter
            counter = 0
            for i in range(1, pair[0]):
                # distance travelled with the current power
                distance = (pair[0] - i) * i
                if distance > pair[1]:
                    counter += 1

            return counter

        data = read_input('6')

        times = [int(x) for x in data[0].split(': ')[1].split(' ') if x != '']
        distances = [int(x) for x in data[1].split(': ')[1].split(' ') if x != '']

        # create tuples of (time, distance)
        pairs = [(times[i], distances[i]) for i in range(len(times))]
        
        result = 11
        for pair in pairs:
            result *= get_distance_travelled(pair)

        print(f"Réponse jour 6 partie 1 : {result}")
    
    def partie2():
        def read_input(puzzle):
            with open(filename, 'r') as f:
                data = f.read().splitlines()
            return data
        
        def get_distance_travelled(pair):
            # exclude 0 and max value as it never makes sense
            # start the counter
            counter = 0
            for i in range(1, pair[0]):
                # distance travelled with the current power
                distance = (pair[0] - i) * i
                if distance > pair[1]:
                    counter += 1

            return counter

        data = read_input('6')
        
        # get the new time
        new_time = data[0].split(': ')[1].split(' ')
        final_time = ""
        for i in new_time:
            final_time += i
        final_time = int(final_time)

        # get the new distance
        new_distance = data[1].split(': ')[1].split(' ')
        final_distance = ""
        for i in new_distance:
            final_distance += i
        final_distance = int(final_distance)
        print(f"Réponse jour 6 partie 2 : {get_distance_travelled((final_time, final_distance))}")
    
    partie1()
    partie2()
    
j6()

Réponse jour 6 partie 1 : 12196800
Réponse jour 6 partie 2 : 36919753


In [63]:
def j7():
    filename = "Input/inputj7.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 7 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 7 partie 2 : ")
    
    partie1()
    partie2()
    
j7()

Réponse jour 7 partie 1 : 
Réponse jour 7 partie 2 : 


In [65]:
def j8():
    filename = "Input/inputj8.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 8 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 8 partie 2 : ")
    
    partie1()
    partie2()
    
j8()

Réponse jour 8 partie 1 : 
Réponse jour 8 partie 2 : 


In [21]:
def j9():
    filename = "Input/inputj9.txt"
        
    def partie1():
        def read_input(puzzle):
            with open(filename, 'r') as f:
                data = f.read().splitlines()
            return data

        data = []
        for line in read_input('9'):
            int_list = [int(i) for i in line.split()]
            data.append(int_list)

        def first_part(int_list):
            pattern = np.array(int_list, dtype=int).reshape(1, -1)

            final_number = 0

            while not np.all(np.diff(pattern) == 0):

                final_number += pattern[-1, -1]
                pattern = np.diff(pattern)

            else:
                final_number += pattern[-1, -1]

            return final_number

        result_1 = 0
        for i in data:
            result_1 += first_part(i)
        print(f"Réponse jour 9 partie 1 : {result_1}")
    
    def partie2():
        def read_input(puzzle):
            with open(filename, 'r') as f:
                data = f.read().splitlines()
            return data

        data = []
        for line in read_input('9'):
            int_list = [int(i) for i in line.split()]
            data.append(int_list)
            
        def second_part(int_list):
            pattern = np.array(int_list, dtype=int).reshape(1, -1)

            final_number = 0 
            sign = 1

            while not np.all(np.diff(pattern) == 0):

                final_number += pattern[0, 0] * sign
                sign *= -1
                pattern = np.diff(pattern)

            else:
                # still add the first number
                final_number += pattern[0, 0] * sign
                sign *= -1

            return final_number

        result_2 = 0
        for i in data:
            result_2 += second_part(i)
        print(f"Réponse jour 9 partie 2 : {result_2}")
    
    partie1()
    partie2()
    
j9()

Réponse jour 9 partie 1 : 1904165718
Réponse jour 9 partie 2 : 964


In [66]:
def j10():
    filename = "Input/inputj10.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 10 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 10 partie 1 : ")
    
    partie1()
    partie2()
    
j10()

Réponse jour 10 partie 1 : 
Réponse jour 10 partie 1 : 


In [67]:
def j11():
    filename = "Input/inputj11.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 11 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 11 partie 1 : ")
    
    partie1()
    partie2()
    
j11()

Réponse jour 11 partie 1 : 
Réponse jour 11 partie 1 : 


In [68]:
def j12():
    filename = "Input/inputj12.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 12 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 12 partie 1 : ")
    
    partie1()
    partie2()
    
j12()

Réponse jour 12 partie 1 : 
Réponse jour 12 partie 1 : 


In [69]:
def j13():
    filename = "Input/inputj13.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 13 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 13 partie 2 : ")
    
    partie1()
    partie2()
    
j13()

Réponse jour 13 partie 1 : 
Réponse jour 13 partie 2 : 


In [70]:
def j14():
    filename = "Input/inputj14.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 14 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 14 partie 2 : ")
    
    partie1()
    partie2()
    
j14()

Réponse jour 14 partie 1 : 
Réponse jour 14 partie 2 : 


In [71]:
def j15():
    filename = "Input/inputj15.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 15 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 15 partie 2 : ")
    
    partie1()
    partie2()
    
j15()

Réponse jour 15 partie 1 : 
Réponse jour 15 partie 2 : 


In [72]:
def j16():
    filename = "Input/inputj16.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 16 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 16 partie 2 : ")
    
    partie1()
    partie2()
    
j16()

Réponse jour 16 partie 1 : 
Réponse jour 16 partie 2 : 


In [73]:
def j17():
    filename = "Input/inputj17.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 17 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 17 partie 2 : ")
    
    partie1()
    partie2()
    
j17()

Réponse jour 17 partie 1 : 
Réponse jour 17 partie 2 : 


In [74]:
def j18():
    filename = "Input/inputj18.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 18 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 18 partie 2 : ")
    
    partie1()
    partie2()
    
j18()

Réponse jour 18 partie 1 : 
Réponse jour 18 partie 2 : 


In [75]:
def j19():
    filename = "Input/inputj19.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 19 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 19 partie 2 : ")
    
    partie1()
    partie2()
    
j19()

Réponse jour 19 partie 1 : 
Réponse jour 19 partie 2 : 


In [76]:
def j20():
    filename = "Input/inputj20.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 20 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 20 partie 2 : ")
    
    partie1()
    partie2()
    
j20()

Réponse jour 20 partie 1 : 
Réponse jour 20 partie 2 : 


In [77]:
def j21():
    filename = "Input/inputj21.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 21 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 21 partie 2 : ")
    
    partie1()
    partie2()
    
j21()

Réponse jour 21 partie 1 : 
Réponse jour 21 partie 2 : 


In [78]:
def j22():
    filename = "Input/inputj22.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 22 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 22 partie 2 : ")
    
    partie1()
    partie2()
    
j22()

Réponse jour 22 partie 1 : 
Réponse jour 22 partie 2 : 


In [79]:
def j23():
    filename = "Input/inputj23.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 23 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 23 partie 2 : ")
    
    partie1()
    partie2()
    
j23()

Réponse jour 23 partie 1 : 
Réponse jour 23 partie 2 : 


In [80]:
def j24():
    filename = "Input/inputj24.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 24 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 24 partie 2 : ")
    
    partie1()
    partie2()
    
j24()

Réponse jour 24 partie 1 : 
Réponse jour 24 partie 2 : 


In [82]:
def j25():
    filename = "Input/inputj25.txt"
    input = []
    with open(filename) as file:
        input = [line.strip() for line in file.readlines()]
        
    def partie1():
        print(f"Réponse jour 25 partie 1 : ")
    
    def partie2():
        print(f"Réponse jour 25 partie 2 : ")
    
    partie1()
    partie2()
    
j25()

Réponse jour 25 partie 1 : 
Réponse jour 25 partie 2 : 
